---
jupyter:
  jupytext:
    formats: md,ipynb
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="06f1218c" -->
# Table des matières
1. [Librairies existantes](#librairies-existantes)
1. [Approche pas à pas](#approche-pas-à-pas)
  1. [Chargement des données](#chargement-des-données)
  1. [Validation de l'équilibre de la distribution des données](#validation-de-léquilibre-de-la-distribution-des-données)
  1. [Les données sont elles MCAR ?](#les-données-sont-elles-mcar-)
  1. [Baseline de prédiction](#ibaselinei-de-prédiction)
  1. [Suppression simple des données manquantes](#suppression-simple-des-données-manquantes)
  1. [Substitution par la moyenne](#substitution-par-la-moyenne)
  1. [Régression déterministe](#régression-déterministe)
  1. [MICE](#mice)
  1. [KNN (données normalisées)](#knn-données-normalisées)
1. [En résumé](#en-résumé)
  1. [Que faut-il comprendre de ce graphique ?](#que-faut-il-comprendre-de-ce-graphique-)
<!-- #endregion -->



In [ ]:
%matplotlib inline

import itertools
import sys
import warnings

import matplotlib.gridspec as gs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import wget
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler

sns.distributions._has_statsmodels = False

warnings.filterwarnings("ignore")


seed = 42
np.random.seed(seed)



<!-- #region id="51b39fe4" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/missing-data-illustration.jpeg"  width="400" />
    <div>
    <font size="0.5">Image Source: https://www.flickr.com/photos/78830297@N05/14556250857</font>
    </div>
</div>

<!-- #endregion -->

<!-- #region id="a758e353" -->
Il arrive souvent que l'on désire analyser un  jeu de données contenant des données manquantes. Le manque
de données est frustrant, mais somme toute, assez courant. Si nous avons généré  le jeu de données, il est possible que
l'information manquante puisse être récupérée en retournant à nos cahiers de laboratoire par exemple. Toutefois,
il arrive que ces données soient irrémédiablement perdues, ou simplement impossibles à obtenir
expérimentalement. Le problème est encore plus sérieux lorsque  le jeu de données provient d'une source externe indépendante.
Il faut alors faire avec, comme on dit.

Imaginez que vous avez un jeu de données contenant 100 observations de trois variables $x_{1}$, $x_{2}$ et $x_{3}$,
et qu'il manque plusieurs entrées dans la colonne $x_{3}$. Si vous ne voulez pas vous
casser la tête et décidez de laisser tomber cette colonne, vous allez perdre $33~\%$ de l'ensemble de vos
données... Imaginez maintenant que pour 25 observations, il manque une entrée ou deux dans
les valeurs des $x_{i}$. Si vous laissez tomber ces lignes, vous allez perdre $25~\%$ de vos observations...
Ce n'est évidemment pas la façon de procéder si l'on est économe de nos données chèrement acquises.

Il serait facile de mettre à zéro toutes ces données manquantes, mais cela pourrait introduire une quantité
importante de biais et réduire l'efficacité des méthodes statistiques utilisées dans l'analyse du jeu de données.

L'imputation des données permet de remplacer les valeurs manquantes dans un jeu de données par des estimations de celles-ci.
Elle utilise les distributions des données présentes et les relations entre les différentes variables $x_{i}$ mesurées.
La figure suivante en montre un exemple. L'imputation prédit la valeur inconnue dans chaque trou de  le jeu de données (en rouge)
en interpolant les valeurs des variables $x_{i}$ mesurées (en vert).

<!-- #endregion -->

<!-- #region id="664e28d5" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/imputation.jpeg"  width="300" />
    <div>
    <font size="0.5">Image Source: https://www.mathworks.com/matlabcentral/fileexchange/60128-sequential-knn-imputation-method</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region colab_type="text" id="d7e6d1aa" -->
> Il est fort probable que vous ne connaissiez pas encore les méthodes de classification utilisées dans ce module.
Il y en a beaucoup en apprentissage automatique et on ne peut pas toutes les expliquer dans cette formation. Elles
sont utilisées ici, car certaines méthodes d'imputation fonctionnent plutôt bien avec celles-ci. La méthode de
validation croisée est également utilisée ici sans explications; elle est présentée dans le module sur la méthodologie.
<!-- #endregion -->

<!-- #region colab_type="text" id="82f03720" -->
# <a id=librairies-existantes>Librairies existantes</a>
<!-- #endregion -->

<!-- #region id="e1127adb" -->
Si vous voulez faire simple : la librairie Scikit-learn offre un [`SimpleImputer`](http://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer) et un [`MissingIndicator`](http://scikit-learn.org/stable/modules/generated/sklearn.impute.MissingIndicator.html#sklearn.impute.MissingIndicator) alors que la librairie Pandas offre la méthode [`fillna()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html).


La librairie [Impyute](https://pypi.org/project/impyute/) offre les fonctionnalités:
- Outils de diagnostic:
     - journaux,
     - distribution des valeurs nulles,
     - comparaison des imputations,
     - test [MCAR](https://www.jstor.org/stable/2290157) de Little. Le test MCAR (*missing completely at random*) évalue via un test d'hypothèse. Les valeurs d'un ensemble de données sont dites MCAR si les évènements qui conduisent à la disparition d'un élément de données particulier sont indépendants à la fois des variables observables et des paramètres d'intérêt non observables, et ce, tout en se produisant de manière aléatoire.
- Imputation de données transversales:
     - imputation aléatoire,
     - k-voisins les plus proches,
     - imputation moyenne,
     - imputation par mode,
     - imputation médiane,
     - imputation multivariée par équations chaînées ([MICE](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/)),
     - espérance/maximisation.
- Imputation de données chronologiques:
     - dernière observation reportée,
     - fenêtre mobile,
     - moyenne mobile intégrée autorégressive (WIP).

Finalement, la librairie [Fancyimpute](https://pypi.org/project/fancyimpute/) offre les fonctionnalités suivantes:
* `SimpleFill`: remplace les entrées manquantes par la moyenne ou la médiane de chaque colonne.
* `KNN`: imputations du voisin le plus proche qui pondère les échantillons en utilisant la différence quadratique moyenne sur les entités pour lesquelles deux lignes contiennent des données observées.
* `SoftImpute`: complétion de la matrice par seuillage souple itératif des décompositions SVD. Inspiré de la librairie [softImpute](https://web.stanford.edu/~hastie/swData/softImpute/vignette.html) pour R, basé sur [Spectral Regularization Algorithms for Learning Large Incomplete Matrices](http://web.stanford.edu/~hastie/Papers/mazumder10a.pdf) de Mazumder et. al..
* `IterativeSVD`: achèvement de la matrice par décomposition itérative SVD de bas rang. Devrait être similaire à SVDimpute de [Missing value estimation methods for DNA microarrays](http://www.ncbi.nlm.nih.gov/pubmed/11395428) de Troyanskaya et. al..
* `IterativeImputer` (ex-MICE): une stratégie pour imputer les valeurs manquantes en modélisant chaque entité avec des valeurs manquantes en fonction des autres entités de manière alternée.
* `MatrixFactorization`: factorisation directe de la matrice incomplète en « U » et « V » de bas rang, avec une pénalité de faible densité L1 sur les éléments de« U »et une pénalité de L2 sur les éléments de « V ». Résolu par descente progressive.
* `NuclearNormMinimization`: implémentation simple de [Exact Matrix Completion via Convex Optimization](http://statweb.stanford.edu/~candes/papers/MatrixCompletion.pdf) d'Emmanuel Candes et Benjamin Recht utilisant [cvxpy](http://www.cvxpy.org). Algorithme trop lent pour les grandes matrices.
* `BiScaler`: estimation itérative de la moyenne des rangées/colonnes et des écarts types pour obtenir une double normalisation. Pas garanti de converger, mais fonctionne bien dans la pratique. Tiré de [Matrix Completion and Low-Rank SVD via Fast Alternating Least Squares](http://arxiv.org/abs/1410.2596).

<!-- #endregion -->

<!-- #region _cell_guid="b57ada49-2e30-473d-9e6e-fa57afd87bc5" _uuid="c66c9a187266122b4fdc5b50ad650988f9e23603" colab_type="text" id="9ce650b0" -->
# <a id=approche-pas-à-pas>Approche pas à pas</a>
<!-- #endregion -->

<!-- #region id="2d635f36" -->
Dans ce qui suit, nous allons appliquer différentes méthodes d'imputation au
 jeu de données Titanic. Celle-ci est tirée d'un projet de classification proposé en 2014 sur la plateforme Web
[Kaggle](https://www.kaggle.com/c/titanic/) qui organise
des compétitions en science des données. Le but est de prédire la survie de passagers lors du
naufrage du Titanic en 1912, au large de Terre-Neuve, au Canada.

 Le jeu de données contient beaucoup de données manquantes. La raison principale étant que lors du voyage inaugural du Titanic,
la compagnie maritime a décidé de remplir ses cales d'émigrants pour les États-Unis afin de maximiser ses profits
et ne pas faire le voyage à moitié vide. Les gens occupant la dernière classe ont embarqué
peu de temps avant leur départ, de la France ou de l'Angleterre, et leurs informations personnelles
n'ont pas toutes été colligées assidument, d'où les données manquantes.
<!-- #endregion -->

<!-- #region id="6d298b6e" -->
## <a id=chargement-des-données>Chargement des données</a>
<!-- #endregion -->

<!-- #region id="387b37dd" -->
Les BD d'entraînement et de test contiennent les variables suivantes:

 - survival: survie (0 = Non, 1 = Oui),
 - pclass: classe de billets (1 = 1er, 2 = 2e, 3 = 3e),
 - sex: sexe,
 - Age: âge en années,
 - sibsp: nombre de frères et sœurs / conjoints à bord du Titanic,
 - parch: nombre de parents/enfants à bord du Titanic,
 - ticket: numéro de billet,
 - name: nom du passager,
 - fare: prix du billet,
 - embarked: port d'embarquement (C = Cherbourg, Q = Queenstown, S = Southampton).
<!-- #endregion -->



In [ ]:
train = pd.read_csv("/pax/shared/GIF-U014/titanic_train.csv")
test = pd.read_csv("/pax/shared/GIF-U014/titanic_test.csv")



<!-- #region colab_type="text" id="1b84e5fa" -->
Certaines variables numériques sont converties en catégories et d'autres sont éliminées, car elles sont inutiles.
<!-- #endregion -->



In [ ]:
# Création des types de colonnes


def enforce_types_titanic(df):
    Pclass_dtype = pd.api.types.CategoricalDtype(categories=[1, 2, 3], ordered=True)
    df.Survived = df.Survived.astype("category")
    df.Pclass = df.Pclass.astype(Pclass_dtype)
    df.Sex = df.Sex.astype("category")
    df.Embarked = df.Embarked.astype("category")
    df = df.drop("Name", axis=1)
    df = df.drop("Ticket", axis=1)
    df = df.drop("index", axis=1)
    df = df.drop("Unnamed: 0", axis=1)
    return df


train = enforce_types_titanic(train)
test = enforce_types_titanic(test)


In [ ]:
# Comptage des valeurs nulles dans le jeu de données


def na_summary(df):
    return df.isnull().sum()


na_summary(train)


In [ ]:
na_summary(test)



<!-- #region id="e0403c53" -->
Il manque des valeurs dans l'ensemble d'entraînement, mais pas dans l'ensemble de test.
<!-- #endregion -->

<!-- #region _cell_guid="602b7065-1c54-4a77-8e47-4836e68a0a05" _uuid="008a9e79ab99b1553bd002d0b745a85fa11f08a2" colab_type="text" id="62d264f3" -->
## <a id=validation-de-léquilibre-de-la-distribution-des-données>Validation de l'équilibre de la distribution des données</a>
<!-- #endregion -->

<!-- #region id="2b75acc6" -->
Si l'on veut entraînement puis tester un classificateur permettant de prédire la survie d'un passager, il
faut que les deux ensembles de données soient distribués similairement selon toutes les variables. Nous allons
comparer les distributions de chaque variable dans les deux ensembles de données.

Est-ce vraiment nécessaire? Pour vous en donner une idée, imaginez que vous travaillez dans une
agence de sondages et que vous voulez prédire le résultat d'une élection. Si vous n'interviewez
que des gens de plus de 65 ans (ensemble entraînement) alors que l'on sait que le jour de l'élection,
des gens de tout âge voteront (ensemble de test) alors vos prédictions initiales risquent d'être
totalement biaisées! Voilà pourquoi il faut que les deux jeux de données soient bien équilibrés.

> À noter que valider la similarité des distributions des caractéristiques entre les deux jeux de données ne doit pas devenir du zèle. Il est normal et désirable d'avoir une "certaine" divergence entre les deux. Nous cherchons surtout à ne pas avoir tous les passagers survivants dans l'ensemble d'entraînement et tous ceux qui ont péri dans l'ensemble de test.
<!-- #endregion -->



In [ ]:
# Calcul de la taille de la grille en fonction du nombre de caractéristiques.
def grid_size(nb_features):
    a = len(nb_features)
    if a % 2 != 0:
        a += 1
    n = np.floor(np.sqrt(a)).astype(np.int64)
    while a % n != 0:
        n -= 1
    m = (a / n).astype(np.int64)
    return m, n


# Affichage des deux distributions pour chaque colonne des dataframes 1 et 2
def dist_comparison(df1, df2):

    assert len(df1.columns) == len(df2.columns)

    m, n = grid_size(df1.columns)
    coords = list(itertools.product(list(range(m)), list(range(n))))

    # Choix des graphiques pour chaque type de colonne.
    numerics = df1.select_dtypes(include=[np.number]).columns
    cats = df1.select_dtypes(include=["category"]).columns

    fig = plt.figure(figsize=(15, 15))
    axes = gs.GridSpec(m, n)
    axes.update(wspace=0.25, hspace=0.25)
    # Graphiques pour données numériques : on fait un KDE de la distribution.
    for i in range(len(numerics)):
        x, y = coords[i]
        ax = plt.subplot(axes[x, y])
        col = numerics[i]

        sns.kdeplot(df1[col].dropna(), ax=ax, label="df1").set(xlabel=col)
        sns.kdeplot(df2[col].dropna(), ax=ax, label="df2")

    # Graphique pour les données catégoriques : diagramme en bâtons.
    for i in range(0, len(cats)):
        x, y = coords[len(numerics) + i]
        ax = plt.subplot(axes[x, y])
        col = cats[i]

        df1_temp = df1[col].value_counts()
        df2_temp = df2[col].value_counts()
        df1_temp = pd.DataFrame(
            {
                col: df1_temp.index,
                "value": df1_temp / len(df1),
                "Set": np.repeat("df1", len(df1_temp)),
            }
        )
        df2_temp = pd.DataFrame(
            {
                col: df2_temp.index,
                "value": df2_temp / len(df2),
                "Set": np.repeat("df2", len(df2_temp)),
            }
        )

        sns.barplot(
            x=col, y="value", hue="Set", data=pd.concat([df1_temp, df2_temp]), ax=ax
        ).set(ylabel="Percentage")


# Affichage de la comparaison entre les données de train et de test (sans la colonne des étiquettes).
dist_comparison(train, test)



<!-- #region _cell_guid="eb108455-8d35-42b5-a6e6-0d644d30959e" _uuid="de2217f045c02397cf908b43a38ee9dc9b86fc6b" colab_type="text" id="95932e6b" -->
Les distributions des variables sont bien balancées (en pourcentages) entre les deux jeux de données.

Comme approche de base (souvent appelée une *baseline*), nous complétons arbitrairement les données manquantes dans l'ensemble d'entraînement

> À noter que l'on ne touche pas à la variable `Age`. On va utiliser plusieurs méthodes d'imputation pour en estimer les valeurs manquantes.
<!-- #endregion -->



In [ ]:
# Embarked est le port d'embarquement, on complète les données manquantes par le premier port.
train.Embarked = train.Embarked.fillna("C")

# Il manque une donnée de prix payé pour le billet; choisissons une valeur arbitraire.
train.Fare = train.Fare.fillna(8.05)


In [ ]:
na_summary(train)


In [ ]:
na_summary(test)



<!-- #region _cell_guid="d36d00c2-6fd3-4b5f-9140-3dff423e0b87" _uuid="f4046a90e27ad0913eca82b47cc76c2d07850ccd" colab_type="text" id="0f25671c" -->
## <a id=les-données-sont-elles-mcar->Les données sont elles MCAR ?</a>
<!-- #endregion -->

<!-- #region id="bbe2261b" -->
L'opération suivante n'est effectuée qu'avec le jeu de données d'entraînement, car il n'y a pas de
valeurs manquantes dans le jeu de données de test.

La variable `Age` est celle pour laquelle on manque le plus de données. Dans ce qui
suit, on sépare les données d'entraînement en deux sous-ensembles; un pour les passagers dont l'âge connu et un
autre pour les passagers dont l'âge est inconnu. On compare ensuite les distributions des autres variables
(hormis les variables `Age` et `Survived`). Si elles sont identiques (MCAR), cela veut dire que les données
manquent complètement au hasard; elles sont distribuées uniformément en fonction du sexe, de la classe de passagers, etc.
Pour vous donner une idée de la pertinence de la question, imaginez par exemple si toutes les valeurs manquantes provenaient
d'hommes de plus de 20 ans ayant voyagé en troisième classe et provenant de Cherbourg. Les valeurs
manquantes correspondraient toutes à un cas particulier, plus difficile à traiter.

Pourquoi faire ce test? Réponse: si les données manquent complètement au hasard, cela veut dire qu'on peut
prédire les âges manquants simplement en interpolant à partir des autres variables.
<!-- #endregion -->



In [ ]:
age_present = train.dropna().drop("Age", 1)
age_missing = train[train.isnull().any(axis=1)].drop("Age", 1)

cat_dtype = pd.api.types.CategoricalDtype(categories=list(range(8)), ordered=True)
age_present.Parch = age_present.Parch.astype(cat_dtype)
age_missing.Parch = age_missing.Parch.astype(cat_dtype)

age_present.SibSp = age_present.SibSp.astype(cat_dtype)
age_missing.SibSp = age_missing.SibSp.astype(cat_dtype)

dist_comparison(age_present.drop("Survived", 1), age_missing.drop("Survived", 1))



<!-- #region _cell_guid="ec01755d-01b8-45e8-b57a-7f83ff302927" _uuid="0fee94f930b9a76b642e4182f2e0a073da427c6c" colab_type="text" id="149b8b44" -->
Il semble que nous ne puissions pas vérifier l’hypothèse MCAR. L'explication semble être que nous sommes
moins susceptibles de connaître l'âge des personnes décédées. Comme en témoigne la proportion beaucoup plus
grande de passagers de la classe inférieure, le pic plus net dans les tarifs plus bas et une légère asymétrie
à l’égard des hommes.

De manière plus significative encore, il semble que les personnes qui se sont embarquées à Queenstown (Q)
ont un taux beaucoup plus élevé d’âge manquant.

> À noter qu'il serait préférable d'utiliser une mesure plus objective et robuste de MCAR, comme le test de Little.
Dans ce qui suit, nous allons toutefois faire l'hypothèse de MCAR afin de simplifier l'analyse.
<!-- #endregion -->

<!-- #region _cell_guid="8be28abb-f498-450a-9d25-4771c432ecec" _uuid="1eb821dba55457db166b63b8bf943bb1f5a30164" colab_type="text" id="446a0186" -->
## <a id=ibaselinei-de-prédiction>*Baseline* de prédiction</a>
<!-- #endregion -->

<!-- #region _cell_guid="64668e8b-0ae7-4c4e-8062-afdaf708c9d4" _uuid="5079fff2dea16b57d9c220e40b694a4dde489b38" colab_type="text" id="149300e6" -->
Sans l'utilisation des données `Age`, notre prédicteur sera un classificateur par forêt aléatoire
avec les paramètres affichés. Toutes les estimations d'erreur de test sont obtenues par une
validation croisée 10 fois. Le classificateur élimine le problème des valeurs manquantes en
éliminant simplement la variable les contenant. Il jette le bébé avec l'eau du bain.

> À noter que la validation croisée est une méthode qui permet, lors de l'entraînement d'un modèle, de partitionner les données d'entraînement en $N$ sections. Le modèle est entrainé sur $N-1$ sections et testé sur la dernière. L'opération est effectuée $N$ fois et on prend la moyenne des $N$ résultats. C'est particulièrement utile lorsque l'ensemble d'entraînement contient peu de données. On s'en sert également pour faire de la sélection de modèles en régression et en classification. La validation croisée est universellement utilisée en apprentissage automatique; on en discute plus en détail dans un des modules sur la méthodologie.
<!-- #endregion -->



In [ ]:
# Préparation des données afin d'enlever les données catégoriques.
def prep_des_donnees(df):
    new_df = df.copy()
    # Classe de cabine en entier
    new_df.Pclass = new_df.Pclass.astype("int")
    # Sexe binaire
    new_df.Sex.cat.categories = [0, 1]
    new_df.Sex = new_df.Sex.astype("int")
    # Port d'embarquement en entier (on pourrait aussi utiliser une variable dummy)
    new_df.Embarked.cat.categories = [0, 1, 2]
    new_df.Embarked = new_df.Embarked.astype("int")
    return new_df


# Même pipeline pour le train et le test.
train_cl = prep_des_donnees(train)
test_cl = prep_des_donnees(test)

# Sélection des colonnes sans l'âge.
Xcol = ["Pclass", "Sex", "SibSp", "Parch", "Fare", "Embarked"]
Ycol = "Survived"
X = train_cl[Xcol]
Y = train_cl[Ycol]

# Mémorisation des jeux de données avant qu'on ne fasse d'autres transformations.
Xbase = X
Ybase = Y

# Classification par forêt aléatoire (Random Forest en anglais, ou RF.)
rf = RandomForestClassifier(n_estimators=1000, max_depth=None, min_samples_split=10)

baseline_err = cross_val_score(rf, X, Y, cv=10, n_jobs=2).mean()
print(
    "[BASELINE] Estimation RF sur Train (n = {}, 10-fold CV): {}".format(
        len(X), baseline_err
    )
)



<!-- #region _cell_guid="9d2a87fb-4317-4fd2-a6db-f5bd62fb3362" _uuid="bcad8d2354eb04f7139567624d27ac2d9e73ad5a" colab_type="text" id="1825ef3d" -->
## <a id=suppression-simple-des-données-manquantes>Suppression simple des données manquantes</a>
<!-- #endregion -->

<!-- #region id="af97c2e1" -->
On ajoute maintenant la variable `Age`, mais en laissant tomber toutes les données pour les passagers dont
l'âge n'est pas connu. On perd, chez ceux-ci, des données pertinentes dans les colonnes autres que celle de l'âge.
<!-- #endregion -->



In [ ]:
Xdel = train_cl.dropna()[Xcol + ["Age"]]
Ydel = train_cl.dropna()[Ycol]

deletion_err = cross_val_score(rf, Xdel, Ydel, cv=10, n_jobs=2).mean()
print(
    "[DELETION] Estimation RF sur Train (n = {}, 10-fold CV): {}".format(
        len(Xdel), deletion_err
    )
)



<!-- #region _cell_guid="a6ac5c2b-364b-476d-8786-dd9f0fd7a9cc" _uuid="4dc1d8e53ddc961c92a334367201c89cdc21af3b" colab_type="text" id="92f49470" -->
## <a id=substitution-par-la-moyenne>Substitution par la moyenne</a>
<!-- #endregion -->

<!-- #region id="0a0fa709" -->
Afin de ne pas perdre inutilement des données, on ne laisse plus tomber les passagers d'âges inconnus. On
remplace les valeurs manquantes d'âge par la moyenne des valeurs connues.
<!-- #endregion -->



In [ ]:
train_cl = prep_des_donnees(train)
train_cl.Age = train_cl.Age.fillna(train_cl.Age.mean(skipna=True))

Xmean = train_cl[Xcol + ["Age"]]
Ymean = train_cl[Ycol]

mean_err = cross_val_score(rf, Xmean, Ymean, cv=10, n_jobs=2).mean()
print(
    "[MEAN] Estimation RF sur Train (n = {}, 10-fold CV): {}".format(
        len(Xmean), mean_err
    )
)



<!-- #region _cell_guid="a52e28e5-1333-4c96-bda7-10b36c39c137" _uuid="ec26ebd70984436e825d406d2616830f0f12e9cc" colab_type="text" id="05416ba4" -->
## <a id=régression-déterministe>Régression déterministe</a>
<!-- #endregion -->

<!-- #region id="933f02f7" -->
Plutôt que de remplacer les âges manquants par l'âge moyen, on modélise la variable `Age` comme une fonction des
autres variables. On entraine, à cette fin, un régresseur n'utilisant que les données des passagers dont l'âge
est connu. On s'en sert ensuite pour prédire l'âge des passagers dont l'âge est inconnu.
<!-- #endregion -->



In [ ]:
train_cl = prep_des_donnees(train)
train_reg = train_cl.dropna()

from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

Xrcol = ["Pclass", "Sex", "SibSp", "Parch", "Fare", "Embarked"]
Yrcol = "Age"

X_reg = train_reg[Xrcol]
Y_reg = train_reg[Yrcol]

age_lm = LinearRegression()
age_lm.fit(X_reg, Y_reg)
abs_residuals = np.absolute(Y_reg - age_lm.predict(X_reg))

# Identifie les indices des passagers dont l'âge est inconnu
nan_inds = train_cl.Age.isnull().to_numpy().nonzero()[0]
train_cl2 = train_cl.copy()

for i in nan_inds:
    train_cl["Age"].at[i] = age_lm.predict(train_cl.loc[i, Xrcol].values.reshape(1, -1))

Xreg = train_cl[Xcol + ["Age"]]
Yreg = train_cl[Ycol]

reg_err = cross_val_score(rf, Xreg, Yreg, cv=10, n_jobs=2).mean()
print(
    "[DETERMINISTIC REGRESSION] Estimation RF sur Train (n = {}, 10-fold CV): {}".format(
        len(Xreg), reg_err
    )
)



<!-- #region _cell_guid="30dc53c5-cb32-4d44-8d23-d2c806a9cd23" _uuid="408dbaa3bdcefb881222e942070e050e36bd40f3" colab_type="text" id="45977539" -->
## <a id=mice>MICE</a>
<!-- #endregion -->

<!-- #region id="66ac5731" -->
La méthode MICE (*multiple imputation by chained equations*) est une méthode d'imputation itérative.
<!-- #endregion -->



In [ ]:
train_cl = prep_des_donnees(train)

X = train_cl.loc[:, Xcol + ["Age"]]
Y = train_cl.loc[:, Ycol]

Xmice = IterativeImputer().fit_transform(X)
Ymice = Y

mice_err = cross_val_score(rf, Xmice, Y, cv=10, n_jobs=2).mean()
print(
    "[MICE] Estimation RF sur Train (n = {}, 10-fold CV): {}".format(
        len(Xmice), mice_err
    )
)



<!-- #region _cell_guid="d37c7551-67bd-4e7f-94d9-25a950021876" _uuid="5cb98a3c43ea86f8348b529623eb2cc39f3970c3" colab_type="text" id="6f16653b" -->
## <a id=knn-données-normalisées>KNN (données normalisées)</a>
<!-- #endregion -->

<!-- #region id="a15ea731" -->
Nous allons maintenant utiliser le classificateur des N plus proches voisins afin de trouver, dans l'espace des variables
$$[\text{Pclass}, \text{Sex}, \text{SibSp}, \text{Parch}, \text{Fare}, \text{Embarked}]$$

quels sont les N passagers, d'âges connus, les plus près
d'un passager d'âge inconnu. On assigne à celui-ci la moyenne des âges de ses $N$ plus proches voisins.

Il peut sembler étrange, dans ce qui suit, de normaliser ces variables puisque plusieurs facteurs (colonnes) correspondent à
des variables binaires ou ordinales. L'idée est de rapporter tous ces facteurs à la même échelle, entre 0 et 1,
afin de calculer correctement les « distances » entre les passagers et trouver leurs plus proches voisins.
Sans la normalisation (ou standardisation), les distances ne dépendraient principalement que des différences de
prix du billet (`Fare`) puisque celui-ci a les plus grandes valeurs.
<!-- #endregion -->



In [ ]:
train_cl = prep_des_donnees(train)

Xcol = ["Pclass", "Sex", "SibSp", "Parch", "Fare", "Embarked"]
X = train_cl.loc[:, Xcol + ["Age"]]
Y = train_cl.loc[:, Ycol]

Xnorm = MinMaxScaler().fit_transform(X)

kvals = np.linspace(1, 100, 20, dtype="int64")

knn_errs = []
for k in kvals:
    knn_err = []
    Xknn = KNNImputer(n_neighbors=k).fit_transform(Xnorm)
    knn_err = cross_val_score(rf, Xknn, Y, cv=10, n_jobs=2).mean()

    knn_errs.append(knn_err)
    print(
        "[KNN] Estimation RF sur Train (n = {}, k = {}, 10-fold CV): {}".format(
            len(Xknn), k, np.mean(knn_err)
        )
    )



<!-- #region id="13c0c463" -->
Affiche le meilleur résultat, c'est-à-dire, pour le nombre $N_{\text{optimal}}$ de voisins.
<!-- #endregion -->



In [ ]:
sns.set_style("darkgrid")
_ = plt.plot(kvals, knn_errs)
_ = plt.xlabel("K")
_ = plt.ylabel("10-fold CV Error Rate")

knn_err = max(knn_errs)
k_opt = kvals[knn_errs.index(knn_err)]

knn = KNNImputer(n_neighbors=k_opt)
knn.fit(Xnorm)
Xknn = knn.transform(Xnorm)

print(
    "[BEST KNN] Estimation RF sur Train (n = {}, k = {}, 10-fold CV): {}".format(
        len(Xknn), k_opt, np.mean(knn_err)
    )
)



<!-- #region _cell_guid="b6b05aaf-e846-4225-95c0-e66f807e61ea" _uuid="c02e770675a39fd1b4c4e3f13639b852f5a24d7d" colab_type="text" id="4772778a" -->
# <a id=en-résumé>En résumé</a>
<!-- #endregion -->

<!-- #region id="ac194a37" -->
Comparons maintenant les résultats des multiples méthodes d'imputation testées (prends quelque temps à exécuter) .
<!-- #endregion -->



In [ ]:
errs = {
    "BEST KNN (k = {})".format(k_opt): knn_err,
    "DETERMINISTIC REGRESSION": reg_err,
    "MICE": mice_err,
    "MEAN": mean_err,
    "DELETION": deletion_err,
    "BASELINE": baseline_err,
}

err_df = pd.DataFrame.from_dict(errs, orient="index")
err_df.index.name = "Imputation Method"
err_df.reset_index(inplace=True)
err_df.columns = ["Imputation", " Estimation sur Test  (10-fold CV)"]

ax = sns.barplot(
    x=err_df.columns[1],
    y=err_df.columns[0],
    order=list.sort(list(errs.values())),
    data=err_df,
)
ax.set_xlabel(err_df.columns[1])
ax.set_ylabel("")
_ = plt.xlim(0.7, 0.8)



<!-- #region id="ad529b45" -->
## <a id=que-faut-il-comprendre-de-ce-graphique->Que faut-il comprendre de ce graphique ?</a>
<!-- #endregion -->

<!-- #region id="57bd887d" -->
Que si on complète les données manquantes avec divers modèles on améliore le pouvoir de
prédiction ... en ajoutant un biais dans les données :-).

Notez à quel point les deux méthodes du bas, bien qu'intuitives, ne sont pas efficaces. La *baseline*
laisse tomber la variable problématique `Age` et la seconde enlève les données des passagers avec
valeurs manquantes. En creusant un peu plus, on est passés d'un score de $74~\%$ à $78~\%$ juste en interpolant
au travers des données existantes.

Un lecteur attentif aura remarqué qu'on n'a pas utilisé les données de test après avoir enlevé les données catégoriques. Le but de ce module était de montrer comment on sélectionne la meilleure méthode d'imputation des données manquantes. Cette opération se fait avec les données d'entraînement.
<!-- #endregion -->
